<a href="https://colab.research.google.com/github/janschill/dye-elo/blob/main/Dyelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Calculate Elo Rating of Dye Players at Tietgen Tournaments

In [239]:
# Calculate elo rating of a team
# Could calculate individual elo rating by using the average elo rating of a team
# https://elosportschallenge.wordpress.com/2017/06/09/individual-ranking-for-doubles-game/
# https://www.geeksforgeeks.org/elo-rating-algorithm/

import math
 
# Function to calculate the Probability
def probability(rating1, rating2):
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))

# K is a constant for giving points away.
# d = 1 Player A wins
# d = 2 Player B wins
# d determines whether Player A wins or Player B.
def elo_rating(rating_player_a, rating_player_b, d):
    K = 50 # points given away
    # To calculate the Winning Probability of Player B
    probability_player_b = probability(rating_player_a, rating_player_b)
    # To calculate the Winning Probability of Player A
    probability_player_a = probability(rating_player_b, rating_player_a)
 
    # Player A wins
    if (d == 1) :
        rpa = rating_player_a + K * (1 - probability_player_a)
        rpb = rating_player_b + K * (0 - probability_player_b) 
    # Player B wins
    else :
        rpa = rating_player_a + K * (0 - probability_player_a)
        rpb = rating_player_b + K * (1 - probability_player_b)

    return [rpa, rpb]
    # print("Updated Ratings:-")
    # print("rating_player_a =", round(rating_player_a, 6)," rating_player_b =", round(rating_player_b, 6))
    
def elo_rating_doubles(team1, team2, d):
  t1p1 = team1[0]
  t1p2 = team1[1]
  t2p1 = team2[0]
  t2p2 = team2[1]
  team1_avg_rating = (t1p1['elo_rating'] + t1p2['elo_rating']) / 2
  team2_avg_rating = (t2p1['elo_rating'] + t2p2['elo_rating']) / 2

  new_elo_team1, new_elo_team2 = elo_rating(team1_avg_rating, team2_avg_rating, d)

  new_rating_t1p1 = t1p1['elo_rating'] + new_elo_team1 - team1_avg_rating
  new_rating_t1p2 = t1p2['elo_rating'] + new_elo_team1 - team1_avg_rating
  new_rating_t2p1 = t2p1['elo_rating'] + new_elo_team2 - team2_avg_rating
  new_rating_t2p2 = t2p2['elo_rating'] + new_elo_team2 - team2_avg_rating

  return [new_rating_t1p1, new_rating_t1p2, new_rating_t2p1, new_rating_t2p2]
# Ra = 1400
# Rb = 1300
# d = 2
# elo_rating(Ra, Rb, d)
# elo_rating_doubles([{ 'elo_rating': Ra } , { 'elo_rating': Ra }], [{ 'elo_rating': Rb }, { 'elo_rating': Rb }], d)

In [240]:
!pip install pychallonge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [241]:
import challonge

challonge.set_credentials("janschill", "")

def challonge_tournament_id(year):
  if year == 2020:
    return "dyeislife"
  if year == 2021:
    return "dyeislifedk2021"

c_t_memo = {}
def challonge_tournament(year):
  id = challonge_tournament_id(year)
  if year not in c_t_memo:
    c_t_memo[year] = challonge.tournaments.show(id)
  return c_t_memo[year]

c_m_memo = {}
def challonge_matches(year):
  if year not in c_m_memo:
    c_m_memo[year] = challonge.matches.index(challonge_tournament(year)['id'])
  return c_m_memo[year]

c_p_memo = {}
def challonge_participants(year):
  if year not in c_p_memo:
    c_p_memo[year] = challonge.participants.index(challonge_tournament(year)['id'])
  return c_p_memo[year]


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('participants').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

In [243]:
def redact_full_name(full_name, level = 1):
  names = full_name.split() # ['Jan', 'Schill']
  if level == 1:
    return names[0]
  else:
    redacted_name = names[0]
    for idx in range(1, len(names), 1):
      redacted_name += " " + names[idx][0]
    return redacted_name

redact_full_name("Jan Schill Sch", 2)

'Jan S S'

In [ ]:
# Convert players to dict
players_raw = []
for idx in range(1, len(rows), 1):
  players_raw.append({
      'id': int(rows[idx][0]),
      'name': redact_full_name(rows[idx][1]),
      'elo_rating': float(rows[idx][4]),
      'tournament_year': int(rows[idx][3]),   
      'team_name': rows[idx][2]
  })

In [245]:
# Combine duplicates; Duplicates are if a players has participated in multiple tournaments
# seen_name = set()
# for obj in l:
#     if obj['name'] not in seen_name:
#         seen_name.add(obj['name'])
#     else:
#         print('Found duplicate', obj['name'])
players = dict()
for p in players_raw:
  id = p['id']
  if id in players:
    players[id]['tournaments'][p['tournament_year']] = p['team_name']
  else:
    players[id] = {
      'id': p['id'],
      'name': p['name'],
      'elo_rating': p['elo_rating'],
      'tournaments': { p['tournament_year']: p['team_name'] }
    }

{'elo_rating': 1400.0,
 'id': 1,
 'name': 'Aksel',
 'tournaments': {2021: 'Mike Dyeson'}}

In [246]:
def strap_and_lower(s):
  return ''.join(e for e in s if e.isalnum()).lower()

def equal_team_names(team_name1, team_name2):
  return strap_and_lower(team_name1) == strap_and_lower(team_name2)

def get_players_by_challonge_team_id(challonge_id, year):
  # gets first element on match
  challonge_team = next((item for item in challonge_participants(year) if item['id'] == challonge_id), None)
  challonge_team_name = challonge_team['name']
  # print(challonge_team_name)
  ps = []
  for item in players.items():
    player = item[1] # items() returns tuple
    if year in player['tournaments'] and equal_team_names(player['tournaments'][year], challonge_team_name):
      # print(f"Found match for {player['name']}")
      ps.append(player)
  
  if ps == []:
    print(f"No match found for {challonge_team_name}")
    ps = [{'elo_rating': 1000, 'name': challonge_team_name + "-1"}, {'elo_rating': 1000, 'name': challonge_team_name + "-2"}]

  return ps

In [248]:
def reset_players_elo():
  for item in players.items():
    player = item[1] # items() returns tuple
    player['elo_rating'] = 1400

def generate_elo(year):
  for match in challonge_matches(year):
    t1p1, t1p2 = get_players_by_challonge_team_id(match['player1_id'], year)
    t2p1, t2p2 = get_players_by_challonge_team_id(match['player2_id'], year)
    
    winner = 1 if match['winner_id'] == match['player1_id'] else 2
    verb = "wins" if winner == 1 else "loses"
    # print(f'{match["player1_id"]} {verb} against {match["player2_id"]}')
    t1p1_new_rating, t1p2_new_rating, t2p1_new_rating, t2p2_new_rating = elo_rating_doubles([t1p1, t1p2],[t2p1, t2p2], winner)
    t1p1['elo_rating'] = t1p1_new_rating
    t1p2['elo_rating'] = t1p2_new_rating
    t2p1['elo_rating'] = t2p1_new_rating
    t2p2['elo_rating'] = t2p2_new_rating

In [249]:
reset_players_elo()

generate_elo(2020)
generate_elo(2021)

No match found for Dyetastic
No match found for ------
No match found for Dyetastic
No match found for ------
No match found for Dyetastic
No match found for Dyetastic


In [250]:
df = pd.DataFrame.from_dict(players, orient='index')
df

,id,name,elo_rating,tournaments
1,1,Aksel,1408.098005,{2021: 'Mike Dyeson'}
2,2,Alberte,1407.127385,{2021: '14x'}
3,3,Alexandre,1350.000000,{2020: 'Les deux merveilles'}
4,4,Anders,1375.053079,{2021: 'Ph.Dye'}
5,5,Andreas,1374.032099,{2021: '3460iditglas'}
...,...,...,...,...
147,147,Will,1416.098676,"{2020: 'Fry up', 2021: 'Isn't it a coincidence..."
148,148,William,1346.426844,{2021: 'Bankers & Wankers'}
149,149,William,1398.484220,{2021: 'Duck Dyeve'}
150,150,William,1534.543568,{2021: 'H.O. Langes helte'}


In [251]:
import json

# result = df.to_json(orient="index")
result = df.to_json(orient='records')[1:-1]
result
# parsed = json.loads(result)
# json.dumps(parsed, indent=4)

'{"id":1,"name":"Aksel","elo_rating":1408.0980051892,"tournaments":{"2021":"Mike Dyeson"}},{"id":2,"name":"Alberte","elo_rating":1407.1273852881,"tournaments":{"2021":"14x"}},{"id":3,"name":"Alexandre","elo_rating":1350.0,"tournaments":{"2020":"Les deux merveilles"}},{"id":4,"name":"Anders","elo_rating":1375.0530785306,"tournaments":{"2021":"Ph.Dye"}},{"id":5,"name":"Andreas","elo_rating":1374.0320992225,"tournaments":{"2021":"3460iditglas"}},{"id":6,"name":"Anna","elo_rating":1376.3192711944,"tournaments":{"2021":"Shine bright like a dyemond"}},{"id":7,"name":"Anna","elo_rating":1351.946208318,"tournaments":{"2021":"Marry rich or dye trying"}},{"id":8,"name":"Asger","elo_rating":1423.2042036527,"tournaments":{"2020":"Live hard, DYE trying"}},{"id":9,"name":"Astrid","elo_rating":1350.0,"tournaments":{"2020":"Dice Dice Baby"}},{"id":10,"name":"Billy","elo_rating":1394.2152140267,"tournaments":{"2020":"Fry up","2021":"Sink-182"}},{"id":11,"name":"Bjorn","elo_rating":1370.4405844409,"tour